In [ ]:
from os import getcwd, chdir, path
import warnings
%load_ext autoreload
import tensorflow as tf
import lstm
warnings.filterwarnings("ignore")

chdir(
    path.join(
        path.expanduser('~'), 
        'PycharmProjects', 
        'DiplomaThesis'
    )
)
print(getcwd())

In [ ]:
dataset = lstm.load_dataset()

In [ ]:
executions = [
    {
        'n_hidden_cells': 128, 
        'learning_rate': 0.005, 
        'alpha': 0.5, 
        'min_epoch_amount': 20,
        'batch_size': 250,
        'dropout_chance': 0.5,
        'model_type': 'gru',
        'optimizer_type': 'adam',
        'use_class_weights': False,
        'train': True
    },
    {
        'n_hidden_cells': 128, 
        'learning_rate': 0.005, 
        'alpha': 0.5, 
        'min_epoch_amount': 20,
        'batch_size': 250,
        'dropout_chance': 0.5,
        'model_type': 'lstm',
        'optimizer_type': 'adam',
        'use_class_weights': False,
        'train': True
    },
    {
        'n_hidden_cells': 128, 
        'learning_rate': 0.005, 
        'alpha': 0.5, 
        'min_epoch_amount': 20,
        'batch_size': 250,
        'dropout_chance': 0.5,
        'model_type': 'rnn',
        'optimizer_type': 'adam',
        'use_class_weights': False,
        'train': True
    }
]

In [3]:
for run in executions:
    
    tf.reset_default_graph()
    
    lstm.run_training(
        dataset=dataset,
        n_hidden_cells=run['n_hidden_cells'], 
        learning_rate=run['learning_rate'], 
        alpha=run['alpha'], 
        min_epoch_amount=run['min_epoch_amount'],
        batch_size=run['batch_size'],
        dropout_chance=run['dropout_chance'],
        model_type=run['model_type'],
        optimizer_type=run['optimizer_type'],
        use_class_weights=run['use_class_weights'],
        train=run['train']
    )

Training:   34500 vectors
Testing:    9500 vectors
Validation: 5500 vectors


Shape X_test: (2500, 25, 423)
Shape Y_test: (2500, 7172)
Shape X_valid: (2500, 25, 423)
Shape Y_valid: (2500, 7172)


25
423
7172
[1024, 1024]



Epoch number 1

Batch number 1



Testing after epoch: 


...

4 : 2
7 : 2
6 : 2
8 : 2
9 : 2
5 : 2
3 : 2
11 : 2
9 : 2
13 : 2
7 : 2
17 : 2
8 : 2
8 : 2
5 : 2
12 : 2
10 : 2
14 : 2
7 : 2
17 : 2
11 : 2
8 : 2
6 : 2
5 : 2
10 : 2
6 : 2
6 : 2
14 : 2
17 : 2
6 : 2
13 : 2
8 : 2
9 : 2
9 : 2
6 : 2
11 : 2
3 : 2
9 : 2
9 : 2
5 : 2
10 : 2
9 : 2
5 : 2
8 : 2
5 : 2
13 : 2
9 : 2
6 : 2
11 : 2
10 : 2
F1 Score:  0.0

Epoch number 12

Batch number 1



Testing after epoch: 


4 : 0
7 : 0
6 : 0
8 : 0
9 : 0
5 : 0
3 : 0
11 : 0
9 : 0
13 : 0
7 : 0
17 : 0
8 : 0
8 : 0
5 : 0
12 : 0
10 : 0
14 : 0
7 : 0
17 : 0
11 : 0
8 : 0
6 : 0
5 : 0
10 : 0
6 : 0
6 : 0
14 : 0
17 : 0
6 : 0
13 : 0
8 : 0
9 : 0
9 : 0
6 : 0
11 : 0
3 : 0
9 : 0
9 : 0
5 : 0
10 : 0
9 : 0
5 : 0
8 : 0
5 : 0
13 : 0
9 : 0
6 : 0
11 : 0
10 : 0
F1 Score:  0.0

Epoch number 13

Batch number 1



Testing after epoch: 


KeyboardInterrupt: 